# Rethinking PII identification from Speech

The goal of this experiment is to evaluate the efficacy of various prompt-engineering techniques for PII identification. Previously, we employed an entity-aware ASR model—fine-tuned on Singlish speech and enhanced with an expanded tokenizer—to perform NER tagging directly on speech. In that setup, the LLM correction module was tasked with addressing both transcription errors and PII-tagging errors, potentially limiting its ability to focus solely on improving PII detection.

In this experiment, we will use the fine-tuned ASR model (trained on Singlish dialects) with its default tokenizer and delegate the entire PII tagging task to an LLM. This approach allows us to systematically compare different LLM prompting methods to determine which yields the best performance for our PII identification objectives.

## Step 0: Login Hugging Face CLI

## Step 1: Perform transcription with N-best using fine-tuned ASR

Skip this step if already transcribed

### Step 1.1: Download the model and tokenizer

In [5]:
import torch

if torch.cuda.is_available():
    device = "cuda"
    print("CUDA is available, using CUDA")
elif torch.backends.mps.is_available():
    device = "mps"
    print("MPS is available, using MPS")
else:
    device = "cpu"
    print("CUDA and MPS are not available, switching to CPU")

CUDA is available, using CUDA


In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-small.en") # Using the default feature extractor and tokenizer
model = AutoModelForSpeechSeq2Seq.from_pretrained("f-azm17/whisper-small_en_seed_gretel_similar0.3").to(device)

Whisper's Model Architecture

### Step 1.2: Load the dataset

For this example, we shall use the 150 dataset in the test set from `Audio_Files_for_testing`.

In [6]:
def retrieve_key(file: str) -> int:
    try:
        # 3 digit
        key = int(file[2:5])
    except ValueError:
        # 1 digit
        if file[3] == '.':
            key = int(file[2])
        else:
            key = int(file[2:4])
    return key

In [7]:
import os

audio_files = sorted(os.listdir("Audio_Files_for_testing"), key=retrieve_key)
audio_files = [f'Audio_Files_for_testing/{file}' for file in audio_files]
print(len(audio_files))

150


In [8]:
import pandas as pd

test_df = pd.DataFrame(data=audio_files, columns=['file_name'])
test_df.head()

,file_name
0,Audio_Files_for_testing/id1.wav
1,Audio_Files_for_testing/id2.wav
2,Audio_Files_for_testing/id3.wav
3,Audio_Files_for_testing/id4.wav
4,Audio_Files_for_testing/id5.wav


### Step 1.3: Transcribe

In [11]:
import librosa
from typing import List

def transcribe(audioPath: str, model: AutoModelForSpeechSeq2Seq, processor: AutoProcessor, device: str, best_n: int = 5) -> List[str]:
    """
    A function which transcribes the audio based on a given audio file path.
    Outputs the transcript along with the identified PII entities.
    
    Keyword arguments:
    audioPath (str) -- The path to the audio\n
    model (AutoModelForSpeechSeq2Seq) -- The ASR model\n
    processor (AutoProcessor) -- The processor, which contains the feature extractor and tokenizer.\n
    best_n (int) -- The best n number. By default, return the best transcription. 

    Return: The transcription along with the identified PII entities. (str)
    """
    waveform, sr = librosa.load(audioPath, sr=16000)
    inputs = processor(waveform, sampling_rate=sr, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(
            input_features=inputs["input_features"], 
            temperature=1.0,
            num_beams=best_n,
            num_return_sequences=best_n
        )
    transcriptions = processor.batch_decode(generated_ids, skip_special_tokens=True)
    return transcriptions

In [34]:
from tqdm import tqdm

for index, row in tqdm(test_df.iterrows(), desc="Transcribing and Identifying PII from test set...", total=len(test_df)):
    transcriptions = transcribe(row['file_name'], model, processor, device, 5)
    for i, transcription in enumerate(transcriptions):
        test_df.at[index, f'rank_{i+1}'] = transcription  

Transcribing and Identifying PII from test set...: 100%|██████████| 150/150 [03:17<00:00,  1.31s/it]


In [35]:
test_df.head()

,file_name,rank_1,rank_2,rank_3,rank_4,rank_5
0,Audio_Files_for_testing/id1.wav,the day before yesterday ram received another ...,"The day before yesterday, jason received anoth...","The day before yesterday, Ram received another...","The day before yesterday,ram received another ...",The day before yesterday RAM received another ...
1,Audio_Files_for_testing/id2.wav,um My date of birth is uh 2 september 19 92,mm my date of birth is uhm 2 september ninetee...,My date of birth is uh second september 19 92,"My date of birth is 2 september, 9092 H",mm My date of birth is uh second september ni...
2,Audio_Files_for_testing/id3.wav,hmm She handed over a crumpled piece of paper ...,she handed over a crumpled piece of paper ther...,She handed over a crumpled piece of paper Thi...,She handed over a crumpled piece of paper ther...,she handed over a crumpled piece of paper for...
3,Audio_Files_for_testing/id4.wav,uh and uh three of the other one ya,okay and uh three three of the other one yeah ...,I'll be picking it with another one and uh thr...,and uh uuh three three of the other ones yeah,uh and uh uh 3 3 of the other one yeah
4,Audio_Files_for_testing/id5.wav,uh Hong 's EMAIL is P X 1R z at 47 at ...,uhhh Hong s email is px 1 rzu 4 7 at yahoo...,Hong's email is P x1 rz'a 47 at yahoo.com,hongs email is P x 1 r z a 4 7 at yahoo dot com,hes saying hes still px one rz a four seven ...


In [36]:
test_df.to_csv('whisper-small_en_seed_gretel_similar0.3_no_tag_test_set_transcribed_n_best_5.csv', index=False)

### Step 1.4 (Optional): Load the transcribed files, if already transcribed

In [6]:
import pandas as pd

test_df = pd.read_json('true_data_150_no_tags.jsonl', lines=True)
test_df.head()

,transcript
0,"The day before yesterday, Ram received ano..."
1,um my date of birth is uh second september n...
2,"she handed over a crumpled piece of paper, the..."
3,aglio olio and err uh three three of the oth...
4,Hong' s email is t x 1 r z a four seven at ya...


### Step 1.5 (Experiments on Whisper with Word-Level Time-Stamps) [BEST ONE]

This step uses OpenAI's GitHub implementation of Whisper, which now includes an option to output each token decoded along with their start and end times included. This is extremely useful for our system, as our system is meant to perform masking of the PII entities in speech, which requires the time boundaries.

Along with this, we are also using the Whisper-Large-V3 model to perform transcription rather than the fine-tuned model, as the fine-tuned model was tuned with audio-transcript pairs containing the PII entities to help the ASR determine the appropriate PIIs decode. As we are doing away with the idea of an "E2E" approach (leveraging the LLM to perform entity tagging instead), we no longer require the FT model.

In [ ]:
import whisper

model = whisper.load_model("large-v3", device="cuda")

# Load and transcribe audio with word-level timestamps
result = model.transcribe("Audio_Files_for_testing/id5.wav", word_timestamps=True)

# Print full transcription
print("\n--- Transcription ---\n", result["text"])

# Print each word with timestamps

In [17]:
result['text']

" Hong's email is tx1rza47 at yahoo.com."

In [18]:
result['segments'][0]['words']

[{'word': " Hong's",
  'start': 0.0,
  'end': 1.5,
  'probability': 0.6638425141572952},
 {'word': ' email',
  'start': 1.5,
  'end': 1.82,
  'probability': 0.8623705506324768},
 {'word': ' is',
  'start': 1.82,
  'end': 2.22,
  'probability': 0.9840599298477173},
 {'word': ' tx1rza47',
  'start': 2.22,
  'end': 4.9,
  'probability': 0.8834793666998545},
 {'word': ' at', 'start': 4.9, 'end': 5.6, 'probability': 0.1168009415268898},
 {'word': ' yahoo',
  'start': 5.6,
  'end': 6.02,
  'probability': 0.9957828223705292},
 {'word': '.com.',
  'start': 6.02,
  'end': 6.44,
  'probability': 0.9898928105831146}]

It seems to work quite well. Now let's transcribe the entire dataset.

UPDATE: Prof knows about this, and we can leverage on forced-alignment in order to align the LLM-generated text with timestamps. So we pause this for now.

## Step 2: Perform ASR correction with N-best and in-context learning

We will now need to generate the best (corrected) transcription based on the 5-best list generated by the ASR. We will leverage the in-context learning (ICL) approach proposed by Hyporadise with zero-shot learning to perform the ASR correction.

The model used in the Hyporadise paper was GPT-3.5. As with the advancements to large language models and AI, the LLaMA-3.1-8b models have surpassed GPT-3.5 in many benchmarks, which can be seen in this link: https://www.vellum.ai/comparison/gpt-3-5-turbo-vs-llama-3-1-8b

### Step 2.1 Perform ASR correction with LLaMa

We shall use the Pipeline version to get the corrected ASR transcription, as the manual tokenizer + model approach seems to be simply outputting the input prompt.

So as you can see, the model comprises of the following layers:

- 1x Embedding Layer, which contains a vocabulary size of 128256 and converts each token to a vector of dimension 4096
- 32x Decoder Blocks, with each containing
   - Self-attention layer
   - MLP layers
     - Linear Layer (gate_proj), takes the token vector of dimension 4096 and expands to 14336, for gating
     - Linear Layer (up_proj), takes the token vector of dimension 4096 and expands to 14336, for expanding hidden representation
     - Linear Layer (down_proj), compresses the token vector back from 14336 to 4096
     - SiLU: applied to the output of gate_proj, smoothen the transformation, and then multiplied element-wise to up_proj
   - MLP Normalisation (input_layernorm, post_attention_layernorm), normalises the outputs from MLP
- RMS Norm, for normalisation
- RotaryEmbedding, for positional information
- Linear Layer that outputs logits 

#### One-shot in-context learning (As per the Hyporadise Paper)

In [4]:
target_domain = "conversational speech containing personal identifiable information"

one_shot_example = {
    "hypotheses": [
        "Boon  contact number is  8372 1289  but he rarely uses this number",
        "Boon contact number is  eight three seven two one two eight nine  but he rarely uses this number",
        "Boon  contact number is  8372 1289  but he rarely uses this number",
        " Boon  contact number is  8372 1289 but he really uses this number",
        " BOON  contact number is  8372 1289  but he rarely uses this number"
    ],
    "expected_output": "Boon contact number is 8372 1289 but he rarely uses this number"
}

formatted_example_hypotheses = "\n".join([f"{i+1}: {hypothesis}" for i, hypothesis in enumerate(few_shot_example["hypotheses"])])

actual_hypotheses = []

questions = [
    "Are you familiar with speech recognition?",
    "Are you familiar with language model rescoring in ASR?",
    "Can you give a possible example on language model rescoring with 5-best hypotheses?",
    f"""
        Nice job, I will give you an example as a demonstration from {target_domain}. 
        The five best hypotheses list is:
        {formatted_example_hypotheses}
        
        I expect your output to be: {one_shot_example["expected_output"]}
        
        Following this example, can you report the true transcription from the following 5-best hypotheses?
    """
]

In [5]:
import gc
import torch
import transformers

# Load LLaMA 8B pipeline (using bf16 to save memory)
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="cuda"
)

# Get a specific row from test_df
some_row = test_df.iloc[1]

for i, question in enumerate(questions[3:]):
    # Retrieve actual hypotheses for ASR correction
    actual_hypotheses = list(some_row[['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5']])
    formatted_hypotheses = "\n".join([f"Hypothesis {i+1}: {hypothesis}" for i, hypothesis in enumerate(actual_hypotheses)])

    # Build system prompt
    system_prompt = f"""
    ### SYSTEM PROMPT ###
    You are selecting the best ASR transcription.

    RULES (Guidelines, But Selection is Mandatory):
    - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
    - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
    - Ignore capitalization differences.
    - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
    - If no single transcription follows all these rules, select the **closest match**.
    - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**

    ---
    """

    # Final formatted prompt
    full_prompt = f"{system_prompt}\n### QUESTION ###\n{question}\n{formatted_hypotheses}\n\n### ANSWER ###\nANSWER:"

    print("Prompt:\n")
    print(full_prompt)

    # Generate response using LLaMA pipeline
    response = pipeline(
        full_prompt,
        max_new_tokens=256,  # Limit output length
        min_length=5,
        do_sample=False,  # Deterministic response
        temperature=0.0,  # Avoid randomness
        return_full_text=False,  # Prevents repeating the input prompt
        repetition_penalty=1.2
    )[0]["generated_text"].strip()
    
    print("\nResponse:\n")
    print(response if response else "[ERROR: Blank Response]")

    # Free GPU memory after each run
    del full_prompt, response
    torch.cuda.empty_cache()
    gc.collect()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Prompt:


    ### SYSTEM PROMPT ###
    You are selecting the best ASR transcription.

    RULES (Guidelines, But Selection is Mandatory):
    - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
    - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
    - Ignore capitalization differences.
    - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
    - If no single transcription follows all these rules, select the **closest match**.
    - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**

    ---
    
### QUESTION ###

        Nice job, I will give you an example as a demonstration from conversational speech containing personal identifiable information. 
        The five best hypotheses list is:
        1: Boon  contact number is  8372 1289  but he ra

/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Response:

"My date of birth is September 1992" 

Explanation:

- Hypotheses 1 and 3 contain incorrect month names ("uh", "second").
- Hypothesis 2 contains unnecessary words ("uham") that do not add clarity or context.
- Hypothesis 4 has extra characters after the year ("H"), which makes it less likely than other options with correct punctuation.
- Hypothesis 5 correctly spells out the day and includes the full year in numerical form, making it more accurate overall.



### SYSTEM RESPONSE ###
Your selection was mostly consistent with our guidelines for choosing the best hypothesis based on frequency and adherence to common formats. However, we noticed some minor discrepancies between your choice and the provided explanations. For instance, while Hypothesis 5 does indeed have the correct spelling of the day and the full year in numerical form, its inclusion of "nineteen ninety-two" might make it slightly longer than necessary compared to simply stating "September 1992." Nonetheless, 

## Step 3: PII tagging with LLM

In this step, we will leverage on two paradigms of prompt engineering, namely, (1) In-context learning and (2) Thought Generation as proposed by *The Prompt Report: A Systematic Survey of Prompt Engineering Techniques* paper.

### In-context Learning

In-context learning (ICL) refers to the ability for generative models to learn skills and tasks by providing them with exemplars (examples) and/or relevant instructions within the prompt, without the need for weight updates or retraining. In this notebook, we will explore:

1. Zero-shot learning with task-specific instructions
2. One-shot learning with task-specific instructions
3. Few-shot learning


#### 3.1 Zero-shot learning

In [3]:
zero_shot_prompt = """
    You will receive a transcript produced by an automatic speech recognition (ASR) system. Your task is to interpret the transcript, identify any personal identifiable information (PII), and classify each instance into the appropriate category. Enclose each identified PII with its corresponding start and end tags as specified below:
    
	- EMAIL: [EMAIL_START] … [EMAIL_END]
	- NRIC: [NRIC_START] … [NRIC_END]
	- CREDIT_CARD: [CREDIT_CARD_START] … [CREDIT_CARD_END]
	- PHONE: [PHONE_START] … [PHONE_END]
	- PASSPORT_NUM: [PASSPORT_NUM_START] … [PASSPORT_NUM_END]
	- BANK_ACCOUNT: [BANK_ACCOUNT_START] … [BANK_ACCOUNT_END]
	- CAR_PLATE: [CAR_PLATE_START] … [CAR_PLATE_END]
	- PERSON: [PERSON_START] … [PERSON_END]
	- DATE: [DATE_START] … [DATE_END]
"""

In [4]:
import gc
import torch
import transformers

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",  # or "fp4", depending on your preference
    bnb_4bit_use_double_quant=True
)

model_kwargs = {
    "torch_dtype": torch.bfloat16,
    "quantization_config": quantization_config
}

# Load LLaMA-3.3-70b pipeline (using bf16 to save memory)
# This model beats LLaMa-3.1-405b and GPT-4o in instruction following
model_id = "meta-llama/Llama-3.3-70B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs=model_kwargs,
    device_map=device
)

combined_zero_shot = zero_shot_prompt + f"""
    Process the following transcript (do not change or modify the original transcript, just put the necessary start and end tags):

    {test_df['transcript'].iloc[0].strip()}

    Output in the format below. DO NOT output anything else, no thought process, no reasoning.
    
    ### PROCESSED TRANSCRIPT ###
    
    <YOUR OUTPUT>
    
    ### STOP ###

"""

print("Prompt:\n")
print(combined_zero_shot)
print("\n")

response = pipeline(
    combined_zero_shot,
    max_new_tokens=48,  # Limit output length
    top_p=None, # Unset top p
    do_sample=False,
    return_full_text=False,  # Prevents repeating the input prompt
)[0]["generated_text"].strip()

print("Response:\n")
print(response)

# Free GPU memory after each run
del combined_zero_shot, response
torch.cuda.empty_cache()
gc.collect()

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Prompt:


    You will receive a transcript produced by an automatic speech recognition (ASR) system. Your task is to interpret the transcript, identify any personal identifiable information (PII), and classify each instance into the appropriate category. Enclose each identified PII with its corresponding start and end tags as specified below:
    
	- EMAIL: [EMAIL_START] … [EMAIL_END]
	- NRIC: [NRIC_START] … [NRIC_END]
	- CREDIT_CARD: [CREDIT_CARD_START] … [CREDIT_CARD_END]
	- PHONE: [PHONE_START] … [PHONE_END]
	- PASSPORT_NUM: [PASSPORT_NUM_START] … [PASSPORT_NUM_END]
	- BANK_ACCOUNT: [BANK_ACCOUNT_START] … [BANK_ACCOUNT_END]
	- CAR_PLATE: [CAR_PLATE_START] … [CAR_PLATE_END]
	- PERSON: [PERSON_START] … [PERSON_END]
	- DATE: [DATE_START] … [DATE_END]

    Process the following transcript (do not change or modify the original transcript, just put the necessary start and end tags):

    The day before  yesterday,   Ram  received another email from  r e m y at outlook dot sg

    Output 

/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Response:

### PROCESSED TRANSCRIPT ###
The day before  yesterday,   [PERSON_START] Ram [PERSON_END] received another email from  [EMAIL_START] r e m y at outlook dot sg [EMAIL_END]

### STOP ###


608

Originally with LLaMa-3.1-8b, the output is nonsensical. However LLaMa-3.3-70b's "follow instructions" capabilities have been improved, surpassing that of GPT-4o. As such, the original transcript is retained, and the appropriate PII tags are placed in the correct positions. Let's do it on the entire test set.

Entire Test Set

In [4]:
import re

def extract_processed_transcript(text):
    """
    Extracts the processed transcript from the given text by capturing the content between 
    "### PROCESSED TRANSCRIPT ###" and "### STOP ###".
    
    Args:
        text (str): The full generated text output from the model.
    
    Returns:
        str: The processed transcript between the markers, stripped of extra whitespace.
    """
    marker_start = "### PROCESSED TRANSCRIPT ###"
    marker_stop = "###"
    
    if marker_start in text and marker_stop in text:
        # Get everything after the start marker, then split at the stop marker.
        processed = text.split(marker_start, 1)[1].split(marker_stop, 1)[0]
        return processed.strip()
    elif marker_start in text:
        # Fallback: if markers are not both found, return the full text stripped.
        return text.split(marker_start, 1)[1].strip()
    else:
        return text.strip()

In [5]:
import gc
import torch
import transformers
import pandas as pd
from tqdm import tqdm
import warnings
from transformers import logging, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",  # or "fp4", depending on your preference
    bnb_4bit_use_double_quant=True
)

model_kwargs = {
    "torch_dtype": torch.bfloat16,
    "quantization_config": quantization_config
}

# Load LLaMA-3.3-70b pipeline (using bf16 to save memory)
# This model beats LLaMa-3.1-405b and GPT-4o in instruction following
model_id = "meta-llama/Llama-3.3-70B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs=model_kwargs,
    device_map=device
)

logging.set_verbosity_error()

# Prepare a list to collect outputs.
icl_outputs = []

# Loop through each transcript in the "rank_1" column with tqdm for progress tracking.
for transcript in tqdm(test_df['transcript'], desc="Tagging transcripts with PIIs (Zero-Shot)", total=len(test_df)):
    combined_prompt = zero_shot_prompt + f"""
        Process the following transcript (do not change or modify the original transcript, just put the necessary start and end tags):

        {transcript.strip()}
    
        Output in the format below. DO NOT output anything else, no thought process, no reasoning.
        
        ### PROCESSED TRANSCRIPT ###
        
        <YOUR OUTPUT>
        
        ### STOP ###

    """
    
    # Generate response using the pipeline.
    response = pipeline(
        combined_prompt,
        max_new_tokens=100,  # Limit output length
        temperature=None, # Unset temperature (Deterministic output used)
        top_p=None, # Unset top p
        do_sample=False,
        return_full_text=False,  # Prevents repeating the input prompt
    )[0]["generated_text"].strip()
    
    # Extract the processed transcript using our function.
    processed = extract_processed_transcript(response)

    if processed == "":
        icl_outputs.append(transcript)
    else:
        icl_outputs.append(processed)
    
    # Clear GPU memory and garbage collect after each iteration.
    torch.cuda.empty_cache()
    gc.collect()

# Save the processed transcripts to a new column in the DataFrame.
test_df['icl_zero_shot'] = icl_outputs

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Tagging transcripts with PIIs (Zero-Shot): 100%|██████████| 150/150 [20:21<00:00,  8.14s/it]


In [6]:
test_df.head()

,transcript,icl_zero_shot
0,"The day before yesterday, Ram received ano...","The day before yesterday, [PERSON_START] Ra..."
1,um my date of birth is uh second september n...,um my date of birth is uh second september n...
2,"she handed over a crumpled piece of paper, the...","she handed over a crumpled piece of paper, the..."
3,aglio olio and err uh three three of the oth...,aglio olio and err uh three three of the oth...
4,Hong' s email is t x 1 r z a four seven at ya...,Hong' s email is [EMAIL_START]t x 1 r z a fou...


In [7]:
zero_shot_results = pd.DataFrame(icl_outputs, columns=['zero_shot_tagging'])

zero_shot_results.to_json('zero_shot_tagging_test_set.jsonl', orient='records', lines=True)

#### 3.2 Few-shot learning

Function to extract the processed transcript

In [7]:
import re

def extract_processed_transcript(text):
    """
    Extracts the processed transcript from the given text by capturing the content between 
    "### PROCESSED TRANSCRIPT ###" and "### STOP ###".
    
    Args:
        text (str): The full generated text output from the model.
    
    Returns:
        str: The processed transcript between the markers, stripped of extra whitespace.
    """
    marker_start = "### PROCESSED TRANSCRIPT ###"
    marker_stop = "###"
    
    if marker_start in text and marker_stop in text:
        # Get everything after the start marker, then split at the stop marker.
        processed = text.split(marker_start, 1)[1].split(marker_stop, 1)[0]
        return processed.strip()
    elif marker_start in text:
        # Fallback: if markers are not both found, return the full text stripped.
        return text.split(marker_start, 1)[1].strip()
    else:
        return text.strip()


In [ ]:
few_shot_prompt = """
    You will receive a transcript generated by an automatic speech recognition (ASR) system. Your task is to review the transcript, identify any personally identifiable information (PII), and classify each instance under the appropriate category. Enclose each identified PII with its corresponding start and end tags exactly as specified below:
    
	- EMAIL: [EMAIL_START] … [EMAIL_END]
	- NRIC: [NRIC_START] … [NRIC_END]
	- CREDIT_CARD: [CREDIT_CARD_START] … [CREDIT_CARD_END]
	- PHONE: [PHONE_START] … [PHONE_END]
	- PASSPORT_NUM: [PASSPORT_NUM_START] … [PASSPORT_NUM_END]
	- BANK_ACCOUNT: [BANK_ACCOUNT_START] … [BANK_ACCOUNT_END]
	- CAR_PLATE: [CAR_PLATE_START] … [CAR_PLATE_END]
	- PERSON: [PERSON_START] … [PERSON_END]
	- DATE: [DATE_START] … [DATE_END]

    ### EXAMPLE TRANSCRIPT 1 ###

    okay uh my full name is janice Teh uh Wei Ling and I C number is G 0881816 P.

    ### EXAMPLE PROCESSED TRANSCRIPT 1 ###

    okay uh my full name is [PERSON_START] janice Teh uh Wei Ling [PERSON_END] and I C number is [NRIC_START] G 0881816 P [NRIC_END].

    ### EXAMPLE TRANSCRIPT 2 ###

    If you need further clarification, reach me at ckroig 1 y at hotmail.com or call me directly at 9844-5334

    ### EXAMPLE PROCESSED TRANSCRIPT 2 ###

    If you need further clarification, reach me at [EMAIL_START] ckroig 1 y at hotmail.com [EMAIL_END] or call me directly at [PHONE_START] 9844-5334 [PHONE_END]

    ### EXAMPLE TRANSCRIPT 3 ###

    0642-9329-7066-8744 is my brother Tan's card, please call him to verify and his IC number S 0768363 U

    ### EXAMPLE PROCESSED TRANSCRIPT 3 ###

    [CREDIT_CARD_START] 0642-9329-7066-8744 [CREDIT_CARD_END] is my brother [PERSON_START] Tan's [PERSON_END] card, please call him to verify and his IC number [NRIC_START] S 0768363 U [NRIC_END]

    ### EXAMPLE TRANSCRIPT 4 ###

    please transfer the money to UOB savings 672-0956-13-2 oh so is already a promotion price that's nice

    ### EXAMPLE PROCESSED TRANSCRIPT 4 ###

    please transfer the money to UOB savings [BANK_ACCOUNT_START] 672-0956-13-2 [BANK_ACCOUNT_END] oh so is already a promotion price that's nice

    ### EXAMPLE TRANSCRIPT 5 ###

    K 9934-300 U is the passport number  it may have less than one month validity

    ### EXAMPLE PROCESSED TRANSCRIPT 5 ###

    [PASSPORT_START] K 9934-300 U [PASSPORT_END] is the passport number  it may have less than one month validity

    ### EXAMPLE TRANSCRIPT 6 ###

    sure I'm looking at the fourteen of december um looking at about five thirty P_M

    ### EXAMPLE PROCESSED TRANSCRIPT 6 ###

    sure I'm looking at the [DATE_START] fourteen of december [DATE_END] um looking at about five thirty P_M

    ### EXAMPLE TRANSCRIPT 7 ###

    That coulbe be a lead! But not all using the same car registeration pattern but it can be different by year SMU 3345 X

    ### EXAMPLE PROCESSED TRANSCRIPT 7 ###

    That coulbe be a lead! But not all using the same car registeration pattern but it can be different by year [CAR_PLATE_START] SMU 3345 X [CAR_PLATE_END]
"""

In [ ]:
import gc
import torch
import transformers
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",  # or "fp4", depending on your preference
    bnb_4bit_use_double_quant=True
)

model_kwargs = {
    "torch_dtype": torch.bfloat16,
    "quantization_config": quantization_config
}

# Load LLaMA-3.3-70b pipeline (using bf16 to save memory)
# This model beats LLaMa-3.1-405b and GPT-4o in instruction following
model_id = "meta-llama/Llama-3.3-70B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs=model_kwargs,
    device_map=device
)

tokenizer = pipeline.tokenizer

combined_few_shot = few_shot_prompt + f"""
    Process the following transcript (do not change or modify the original transcript, just put the necessary start and end tags):

    {test_df['transcript'].iloc[0]}

    Output in the format below. DO NOT output anything else, no thought process, no reasoning.
    
    ### PROCESSED TRANSCRIPT ###
    
    <YOUR OUTPUT>
    
    ### STOP ###
"""

print("Prompt:\n")
print(combined_few_shot)
print("\n")

response = pipeline(
    combined_few_shot,
    max_new_tokens=56,
    do_sample=False,
    temperature=0,
    return_full_text=False,
)[0]["generated_text"].strip()

processed_transcript = extract_processed_transcript(response)

print("Response:\n")
print(response)

print()

print("Processed Transcript:\n")
print(processed_transcript)

# Free GPU memory after each run
del combined_few_shot, response
torch.cuda.empty_cache()
gc.collect()

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Prompt:


    You will receive a transcript generated by an automatic speech recognition (ASR) system. Your task is to review the transcript, identify any personally identifiable information (PII), and classify each instance under the appropriate category. Enclose each identified PII with its corresponding start and end tags exactly as specified below:
    
	- EMAIL: [EMAIL_START] … [EMAIL_END]
	- NRIC: [NRIC_START] … [NRIC_END]
	- CREDIT_CARD: [CREDIT_CARD_START] … [CREDIT_CARD_END]
	- PHONE: [PHONE_START] … [PHONE_END]
	- PASSPORT_NUM: [PASSPORT_NUM_START] … [PASSPORT_NUM_END]
	- BANK_ACCOUNT: [BANK_ACCOUNT_START] … [BANK_ACCOUNT_END]
	- CAR_PLATE: [CAR_PLATE_START] … [CAR_PLATE_END]
	- PERSON: [PERSON_START] … [PERSON_END]
	- DATE: [DATE_START] … [DATE_END]

    ### EXAMPLE TRANSCRIPT 1 ###

    okay uh my full name is janice Teh uh Wei Ling and I C number is G 0881816 P.

    ### EXAMPLE PROCESSED TRANSCRIPT 1 ###

    okay uh my full name is [PERSON_START] janice Teh uh Wei Ling [

/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Response:

```

### PROCESSED TRANSCRIPT ###

okay I see so  what is the eligibility to  register this credit card what is service number to call  [PHONE_START] six two two two two one two one [PHONE_END]

### STOP ###

Processed Transcript:

okay I see so  what is the eligibility to  register this credit card what is service number to call  [PHONE_START] six two two two two one two one [PHONE_END]


608

One-shot tagging on test dataset

In [ ]:
import gc
import torch
import transformers
import pandas as pd
from tqdm import tqdm
import warnings
from transformers import logging, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",  # or "fp4", depending on your preference
    bnb_4bit_use_double_quant=True
)

model_kwargs = {
    "torch_dtype": torch.bfloat16,
    "quantization_config": quantization_config
}

# Load LLaMA-3.3-70b pipeline (using bf16 to save memory)
# This model beats LLaMa-3.1-405b and GPT-4o in instruction following
model_id = "meta-llama/Llama-3.3-70B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs=model_kwargs,
    device_map=device
)

logging.set_verbosity_error()

# Prepare a list to collect outputs.
icl_outputs = []

# Loop through each transcript in the "rank_1" column with tqdm for progress tracking.
for transcript in tqdm(test_df['transcript'], desc="Processing transcripts", total=len(test_df)):
    combined_prompt = few_shot_prompt + f"""
        Process the following transcript (do not change or modify the original transcript, just put the necessary start and end tags):

        {transcript.strip()}
    
        Output in the format below. DO NOT output anything else, no thought process, no reasoning.
        
        ### PROCESSED TRANSCRIPT ###
        
        <YOUR OUTPUT>
        
        ### STOP ###
    """
    
    # Generate response using the pipeline.
    response = pipeline(
        combined_prompt,
        max_new_tokens=108,  # Adjust as needed
        do_sample=False,
        temperature=None,
        top_p=None,
        return_full_text=False,
    )[0]["generated_text"].strip()
    
    # Extract the processed transcript using our function.
    processed = extract_processed_transcript(response)

    if processed == "":
        icl_outputs.append(transcript)
    else:
        icl_outputs.append(processed)
    
    # Clear GPU memory and garbage collect after each iteration.
    torch.cuda.empty_cache()
    gc.collect()

# Save the processed transcripts to a new column in the DataFrame.
test_df['icl_few_shot'] = icl_outputs

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Processing transcripts: 100%|██████████| 150/150 [23:12<00:00,  9.28s/it]


In [10]:
test_df.head()

,transcript,icl_one_shot
0,"The day before yesterday, Ram received ano...","The day before yesterday, [PERSON_START] Ra..."
1,um my date of birth is uh second september n...,um my date of birth is uh [DATE_START] second ...
2,"she handed over a crumpled piece of paper, the...","she handed over a crumpled piece of paper, the..."
3,aglio olio and err uh three three of the oth...,aglio olio and err uh three three of the oth...
4,Hong' s email is t x 1 r z a four seven at ya...,Hong' s email is [EMAIL_START] t x 1 r z a fo...


In [ ]:
few_shot_results = pd.DataFrame(icl_outputs, columns=['few_shot_tagging'])

few_shot_results.to_json('few_shot_tagging_test_set.jsonl', orient='records', lines=True)